In [ ]:
import matplotlib.pyplot as plt
import sys
sys.path.append('../python_scripts')

import mesa as ms
import pylab as p
import numpy as np


%matplotlib inline 
plt.rcParams['figure.figsize'] = 12, 6  # that's default image size for this interactive session

data_path = "/Users/tassos/repos/CE_mesa/working/LOGS/"
data_path= "/data/disk1/fragkos/mesa_projects/CE/DVU_Mexico/8M_30M_1400R/step2_macleod_setE2_noMLT_noFL/LOGS/"

a = ms.mesa(data_path=data_path, parallel=True, abundances=False, log_abundances = True, Yaxis='mass', Xaxis="log_inv_star_age",czones=False, Variable='v_div_vesc', orbit=True)

print len(a.profiles)

In [ ]:
i=len(a.profiles)-1



if (not "luminosity" in a.profiles[i].dtype.names):
    try:
        a.profiles[i] = np.lib.recfunctions.append_fields(a.profiles[i],'luminosity',
                        data = 10**(a.profiles[i]['logL']), asrecarray=True)
        a.profiles[i-1] = np.lib.recfunctions.append_fields(a.profiles[i-1],'luminosity',
                        data = 10**(a.profiles[i]['logL']), asrecarray=True)

        a.profiles[i-2] = np.lib.recfunctions.append_fields(a.profiles[i-2],'luminosity',
                        data = 10**(a.profiles[i]['logL']), asrecarray=True)

    except Exception:
        raise ValueError("Column 'logL' and 'luminosity' is missing from the profile files")


Lsun = 3.828e33
secyer = 24.0*3600.0*365.24
GGG = 6.674e-8
Msun = 1.989e33
Rsun = 6.955e10
clight = 29979245800
pi = 3.1415
crad = 7.57e-15



def make_patch_spines_invisible(ax):
    ax.set_frame_on(True)
    ax.patch.set_visible(False)
    for sp in ax.spines.values():
        sp.set_visible(False)

fig, host = plt.subplots()
fig.subplots_adjust(right=0.75)


par1 = host.twinx()

# par2 = host.twinx()
# par2.spines["right"].set_position(("axes", 1.2))
# make_patch_spines_invisible(par2)
# par2.spines["right"].set_visible(True)

par3 = host.twinx()
par3.spines["right"].set_position(("axes", 1.3))
make_patch_spines_invisible(par3)
par3.spines["right"].set_visible(True)

par4 = host.twinx()
par4.spines["right"].set_position(("axes", 1.1))
make_patch_spines_invisible(par4)
par4.spines["right"].set_visible(True)

par5 = host.twinx()
par5.spines["right"].set_position(("axes", 1.4))
make_patch_spines_invisible(par5)
par5.spines["right"].set_visible(True)


par6 = host.twinx()
par6.spines["right"].set_position(("axes", 1.5))
make_patch_spines_invisible(par6)
par6.spines["right"].set_visible(True)

par7 = host.twinx()
par7.spines["right"].set_position(("axes", 1.6))
make_patch_spines_invisible(par7)
par7.spines["right"].set_visible(True)




#i=6
#i=0

# Set x-variable
x = a.profiles[i]["zone"]
# x = 10**a.profiles[i]["logR"]
# x = a.profiles[i]["logR"]
# x = a.profiles[i]["mass"]


# x = a.profiles[i]["q"]
# xmin = 0
# xmax = 200
xmin = min(a.profiles[i]["zone"])
xmax = max(a.profiles[i]["zone"])-1



tau = 10.**a.profiles[i]["logtau"]


# tau_zone = np.max([np.where(tau < 0.15)])
# tau_x = x[tau_zone]
# plt.axvline(tau_x, color='gray', linewidth=3, alpha=0.2, linestyle='--')



tau_zone = np.max([np.where(tau < 1.0)])
tau_x = x[tau_zone]
plt.axvline(tau_x, color='gray', linewidth=3, alpha=0.2)

tau_zone = np.max([np.where(tau < 10)])
tau_x = x[tau_zone]
plt.axvline(tau_x, color='gray', linewidth=3, alpha=0.4)

tau_zone = np.max([np.where(tau < 100)])
tau_x = x[tau_zone]
plt.axvline(tau_x, color='gray', linewidth=3, alpha=0.6)


# alfa = s% dq(j-1)/(s% dq(j-1) + s% dq(j))
# alfa = a.profiles[i]["dq"][1:]/(a.profiles[i]["dq"][1:]+a.profiles[i]["dq"][:-1])
# kap_face = alfa*a.profiles[i]["opacity"][:-1] + (1-alfa)*a.profiles[i]["opacity"][1:]
#          kap_face = alfa*s% opacity(j) + (1d0 - alfa)*s% opacity(j-1)

alfa = a.profiles[i]["dq"][:-1]/(a.profiles[i]["dq"][:-1]+a.profiles[i]["dq"][1:])
kap_face = alfa*a.profiles[i]["opacity"][1:] + (1-alfa)*a.profiles[i]["opacity"][:-1]




area = 4.*pi*(10**a.profiles[i]["logR"][1:]*Rsun)**2
dT4dm = ((10**a.profiles[i]["logT"][:-1])**4-(10**a.profiles[i]["logT"][1:])**4)/a.profiles[i]["dm"][:-1]
dT4dm = (4*(10**a.profiles[i]["logT"][:-1])**3) * (np.gradient(10**a.profiles[i]["logT"][:-1], edge_order=2))/(np.gradient(a.profiles[i]["mass"][:-1]*Msun, edge_order=2))

Lrad = -area*area*crad*clight/(3*kap_face)*dT4dm/Lsun
Lrad = np.append(Lrad, Lrad[-1])



print(Lrad, a.profiles[i]["luminosity"],(4*(10**a.profiles[i]["logT"][:-1])**3),np.gradient(10**a.profiles[i]["logT"][:-1]),np.gradient(a.profiles[i]["mass"][:-1]*Msun))

#fig = plt.figure(figsize=(10,6))

logD, = host.plot(x[xmin:xmax],a.profiles[i]["logRho"][xmin:xmax], label="log Density", color='b')
lum, = par1.plot(x[xmin:xmax],a.profiles[i]["luminosity"][xmin:xmax], label="Luminosity", color='r')
# Lrad, = par1.plot(x[xmin:xmax],Lrad[xmin:xmax], label="Luminosity", color='r', linestyle='dotted')

# eps_visc, = par2.plot(x,a.profiles[i]["eps_visc"], label=r"$\epsilon_{\rm visc}$", color='g', linestyle='solid')
# eps_grav, = par2.plot(x,a.profiles[i]["eps_grav"], label=r"$\epsilon_{\rm grav}$", color='g', linestyle='dashed')
# eps_nuc, = par2.plot(x,a.profiles[i]["eps_recombination"], label=r"$\epsilon_{\rm rec}$", color='g', linestyle='dotted')
vel, = par3.plot(x[xmin:xmax],a.profiles[i]["velocity"][xmin:xmax]/1.0e5, label="velocity", color='m')
cs, = par3.plot(x[xmin:xmax],a.profiles[i]["csound"][xmin:xmax]/1.0e5, label="Sound speed", color='m', linestyle='dashed')
v_esc = np.sqrt(2.*GGG*a.profiles[i]["mass"]*Msun/(10**a.profiles[i]["logR"]*Rsun))/1.0e5
vesc, = par3.plot(x[xmin:xmax],v_esc[xmin:xmax], label="escape velocity", color='m', linestyle='dotted')

temp, = par4.plot(x[xmin:xmax],a.profiles[i]["logT"][xmin:xmax], label="log Temp", color='y')
opacity, = par5.plot(x[xmin:xmax],np.log10(a.profiles[i]["opacity"][xmin:xmax]), label="opacity", color='c')
#brunt_N2, = par6.plot(x, a.profiles[i]["brunt_N2"], label="Brunt-Vaisala", color='k')
# e_grav, = par6.plot(x, a.profiles[i]["mass"]/10**a.profiles[i]["logR"], label="e_grav", color='k')
mass, = par6.plot(x[xmin:xmax], a.profiles[i]["mass"][xmin:xmax], label="Mass coordinate", color='k')
radius, = par7.plot(x[xmin:xmax], 10**a.profiles[i]["logR"][xmin:xmax], label="Radius coordinate", color='grey')




# Label y-axes
host.set_ylabel('Log Density')
par1.set_ylabel('Luminosity')
# par2.set_ylabel('eps')
par3.set_ylabel('velocity (km/s)')
par4.set_ylabel('log Temp')
par5.set_ylabel('Opacity')
#par6.set_ylabel('Brunt-Vaisala')
par6.set_ylabel('Mass Coordinate (Msolar)')
par7.set_ylabel('Radius coordinate (Rsolar)')



# Set y-axis label colors
host.yaxis.label.set_color(logD.get_color())
par1.yaxis.label.set_color(lum.get_color())
# par2.yaxis.label.set_color(eps_grav.get_color())
par3.yaxis.label.set_color(vel.get_color())
par4.yaxis.label.set_color(temp.get_color())
par5.yaxis.label.set_color(opacity.get_color())
#par6.yaxis.label.set_color(brunt_N2.get_color())
par6.yaxis.label.set_color(mass.get_color())
par7.yaxis.label.set_color(radius.get_color())


# # Y-limit
# host.set_ylim(min(a.profiles[i]["logRho"][xmin:xmax]), max(a.profiles[i]["logRho"][xmin:xmax]))



# par4.set_ylim(min(a.profiles[i]["logT"][xmin:xmax]), max(a.profiles[i]["logT"][xmin:xmax]))
# y_min_v = min([min(a.profiles[i]["velocity"][xmin:xmax]/1.0e5),min(a.profiles[i]["csound"][xmin:xmax]/1.0e5),min(v_esc[xmin:xmax])])
# y_max_v = max([max(a.profiles[i]["velocity"][xmin:xmax]/1.0e5),max(a.profiles[i]["csound"][xmin:xmax]/1.0e5),max(v_esc[xmin:xmax])])
# par3.set_ylim(y_min_v,y_max_v)
# par7.set_ylim(min(a.profiles[i]["entropy"][xmin:xmax]), max(a.profiles[i]["entropy"][xmin:xmax]))
# print("entropy",a.profiles[i]["entropy"][xmin:xmax])


#par6.set_ylim(0.0, 0.5)


# Set y-axis tick colors
tkw = dict(size=4, width=1.5)
host.tick_params(axis='y', colors=logD.get_color(), **tkw)
par1.tick_params(axis='y', colors=lum.get_color(), **tkw)
# par2.tick_params(axis='y', colors=eps_grav.get_color(), **tkw)
par3.tick_params(axis='y', colors=vel.get_color(), **tkw)
par4.tick_params(axis='y', colors=temp.get_color(), **tkw)
par5.tick_params(axis='y', colors=opacity.get_color(), **tkw)
#par6.tick_params(axis='y', colors=brunt_N2.get_color(), **tkw)
par6.tick_params(axis='y', colors=mass.get_color(), **tkw)
par7.tick_params(axis='y', colors=radius.get_color(), **tkw)

host.tick_params(axis='x', **tkw)

# Set x-limits
plt.xlim(x[xmin], x[xmax])


# Legend
# lines = [logD, lum, Lrad, temp, eps_visc, eps_nuc, vel, cs,vesc, opacity]
lines = [logD, lum, Lrad, temp, vel, cs,vesc, opacity]


#host.legend(lines, [l.get_label() for l in lines], loc=2)

plt.tight_layout()

plt.show()

# Just luminosity plot
fig, host = plt.subplots(figsize=(11.2,3))
plt.title(r"Flux / ($\rho_{E}$\times c)")
R= 10**a.profiles[i]["logR"]
dR = R
dR[:-1] = R[:-1]-R[1:]
dR[-1] = dR[-2]
F = a.profiles[i]["luminosity"] * Lsun / (4.*pi*R**2)
rho_E = a.profiles[i]["energy"] * a.profiles[i]["dm"]/(4./3.*pi*((R+dR)**3-R**3))
plt.plot(a.profiles[i]["zone"], F/(rho_E*clight), color='r')


# plt.plot(a.profiles[i]["zone"], a.profiles[i]["luminosity"]*2.07e-13*secyer*Lsun, color='r')
# plt.plot(a.profiles[i-1]["zone"], a.profiles[i-1]["luminosity"]*1.73e-13*secyer*Lsun, color='r', linestyle='--')
# plt.plot(a.profiles[i-2]["zone"], a.profiles[i-2]["luminosity"]*2.88e-13*secyer*Lsun, color='r', linestyle=':')
plt.xlim(xmin,xmax)
# plt.ylim(-2.0e35, 2.0e35)
plt.show()


# # Luminosity and energy
# fig, host = plt.subplots(figsize=(11.2,3))
# plt.title('Specific Energy (erg/g)')

# plt.plot(a.profiles[i]["zone"], np.abs(a.profiles[i]["luminosity"]*2.07e-13*secyer*Lsun/a.profiles[i]["dm"]), color='r', label="Luminosity")
# plt.plot(a.profiles[i-1]["zone"], np.abs(a.profiles[i-1]["luminosity"]*1.73e-13*secyer*Lsun/a.profiles[i-1]["dm"]), color='r', linestyle='--')
# plt.plot(a.profiles[i-2]["zone"], np.abs(a.profiles[i-2]["luminosity"]*2.88e-13*secyer*Lsun/a.profiles[i-2]["dm"]), color='r', linestyle=':')
# plt.plot(a.profiles[i]["zone"], abs(a.profiles[i]["energy"]), color='b', label="Internal")
# plt.plot(a.profiles[i]["zone"], 0.5*a.profiles[i]["velocity"]**2, color='g', label="Kinetic")
# plt.plot(a.profiles[i]["zone"], GGG*a.profiles[i]["mass"]*Msun/10**a.profiles[i]["logR"]/Rsun, color='orange', label="-Gravitational")
# plt.xlim(xmin,xmax)
# plt.yscale('log')
# plt.legend()
# plt.ylim(1e4, 1e15)
# #plt.ylim(0.9*min(a.profiles[i]["luminosity"][xmin:xmax]), 1.05*max(a.profiles[i]["luminosity"][xmin:xmax]))
# plt.show()


# Grid cell mass plot
fig, host = plt.subplots(figsize=(11.2,3))
plt.title('Grid Cell Fractional Mass (dq)')
plt.plot(a.profiles[i]["zone"], a.profiles[i]["dq"], color='r', linestyle='-')
plt.plot(a.profiles[i-1]["zone"], a.profiles[i-1]["dq"], color='r', linestyle='--')
plt.plot(a.profiles[i-2]["zone"], a.profiles[i-2]["dq"], color='r', linestyle=':')
plt.xlim(xmin,xmax)
plt.ylim(0.9*min(a.profiles[i]["dq"][xmin:xmax]), 1.1*max(a.profiles[i]["dq"][xmin:xmax]))
plt.yscale('log')
plt.show()





# # Mechanical work plot
# fig, host = plt.subplots(figsize=(11.2,3))
# plt.title('Mechanical Work (erg)')
# plt.plot(a.profiles[i]["zone"], 10**a.profiles[i]["logP"]*8.0*np.pi*(10**a.profiles[i]["logR"]*Rsun)*a.profiles[i]["dr"]*a.profiles[i]["velocity"], color='b')
# plt.plot(a.profiles[i-1]["zone"], 10**a.profiles[i-1]["logP"]*4.0*np.pi*(10**a.profiles[i-1]["logR"]*Rsun)**2, color='b', linestyle='--')
# plt.plot(a.profiles[i-2]["zone"], 10**a.profiles[i-2]["logP"]*4.0*np.pi*(10**a.profiles[i-2]["logR"]*Rsun)**2, color='b', linestyle=':')
# plt.xlim(xmin,xmax)
# plt.ylim(0.9*min(a.profiles[i]["logP"][xmin:xmax]), 1.05*max(a.profiles[i]["logP"][xmin:xmax]))
# plt.show()



# Temperature plot
fig, host = plt.subplots(figsize=(11.2,3))
plt.title('Temperature (K)')
plt.plot(10.0**a.profiles[i]["logR"], 10.0**a.profiles[i]["logT"], color='b')
plt.plot(10.0**a.profiles[i-1]["logR"], 10.0**a.profiles[i-1]["logT"], color='b', linestyle='--')
plt.plot(10.0**a.profiles[i-2]["logR"], 10.0**a.profiles[i-2]["logT"], color='b', linestyle=':')
#plt.xlim(10.0**a.profiles[i]["logR"][xmin],10.0**a.profiles[i]["logR"][xmax])
plt.xlim(113.2, 112.8)
plt.ylim(0.9*min(10.0**a.profiles[i]["logT"][xmin:xmax]), 1.05*max(10.0**a.profiles[i]["logT"][xmin:xmax]))
plt.show()



# Temperature plot
fig, host = plt.subplots(figsize=(11.2,3))
plt.title('Temperature (K)')
plt.plot(a.profiles[i]["zone"], 10.0**a.profiles[i]["logT"], color='b')
plt.plot(a.profiles[i-1]["zone"], 10.0**a.profiles[i-1]["logT"], color='b', linestyle='--')
plt.plot(a.profiles[i-2]["zone"], 10.0**a.profiles[i-2]["logT"], color='b', linestyle=':')
plt.xlim(xmin,xmax)
plt.ylim(0.9*min(10.0**a.profiles[i]["logT"][xmin:xmax]), 1.05*max(10.0**a.profiles[i]["logT"][xmin:xmax]))
plt.show()

# Pressure plot
# fig, host = plt.subplots(figsize=(11.2,3))
# plt.title('Pressure')
# plt.plot(a.profiles[i]["zone"], a.profiles[i]["pressure"], color='k', alpha=0.2, linewidth=8.0)
# plt.plot(a.profiles[i]["zone"], a.profiles[i]["pgas"], color='b')
# plt.plot(a.profiles[i]["zone"], a.profiles[i]["prad"], color='r')
# #plt.plot(a.profiles[i-1]["zone"], a.profiles[i-1]["pressure"], color='k', linestyle='--')
# #plt.plot(a.profiles[i-2]["zone"], a.profiles[i-2]["pressure"], color='k', linestyle=':')
# plt.xlim(xmin,xmax)
# plt.ylim(0.9*min(a.profiles[i]["pressure"][xmin:xmax]), 1.05*max(a.profiles[i]["pressure"][xmin:xmax]))
# plt.show()





# Blackbody radiation plot
fig, host = plt.subplots(figsize=(11.2,3))
plt.title('Blackbody radiation (erg/s)')
sf_const = 5.6704e-5
plt.plot(a.profiles[i]["zone"], 4.0*np.pi*sf_const*(10**a.profiles[i]["logR"] * Rsun)**2*(10**a.profiles[i]["logT"])**4, color='b')
plt.plot(a.profiles[i-1]["zone"], 4.0*np.pi*sf_const*(10**a.profiles[i-1]["logR"] * Rsun)**2*(10**a.profiles[i-1]["logT"])**4, color='b', linestyle='--')
plt.plot(a.profiles[i-2]["zone"], 4.0*np.pi*sf_const*(10**a.profiles[i-2]["logR"] * Rsun)**2*(10**a.profiles[i-2]["logT"])**4, color='b', linestyle=':')
plt.xlim(xmin,xmax)
plt.ylim(3e36, 1e37)
plt.show()



# Internal energy
fig, host = plt.subplots(figsize=(11.2,3))
plt.title('Internal Energy (erg)')
plt.plot(a.profiles[i]["zone"], a.profiles[i]["energy"]*a.profiles[i]["dm"], color='b')
#plt.ylim(0.9*min(10.0**a.profiles[i]["energy"][xmin:xmax]), 1.05*max(10.0**a.profiles[i]["energy"][xmin:xmax]))
plt.xlim(xmin,xmax)
plt.ylim(1e38, 1e40)
plt.yscale('log')
plt.show()



# Mass
fig, host = plt.subplots(figsize=(11.2,3))
plt.title('Mass exterior to zone (Msun)')
plt.plot(a.profiles[i]["zone"], a.profiles[i]["mass"][0]-a.profiles[i]["mass"], color='k')
plt.xlim(xmin,xmax)
plt.ylim(1.0e-7, 1.0e-4)
plt.yscale('log')
plt.show()



# Now, let's plot the ionization states of H, He

fig, host = plt.subplots(figsize=(11.2,3))
fig.subplots_adjust(right=0.75)

#par1 = host.twinx()
plt.plot(x,a.profiles[i]["neutral_fraction_H"], label="H fraction", color='r')
plt.plot(x,1.0 - a.profiles[i]["neutral_fraction_H"], label="H+ fraction", color='r', linestyle='dashed')
plt.plot(x,a.profiles[i]["neutral_fraction_He"], label="He fraction", color='b')
frac_He_II = 2.0 - 2.0 * a.profiles[i]["neutral_fraction_He"] - a.profiles[i]["avg_charge_He"]
frac_He_III = 1.0 - a.profiles[i]["neutral_fraction_He"] - frac_He_II
plt.plot(x,frac_He_II, label="He+ fraction", color='b', linestyle='dashed')
plt.plot(x,frac_He_III, label="He++ fraction", color='b', linestyle='dotted')

plt.plot(x,a.profiles[i]["neutral_fraction_C"], label="C fraction", color='m')
plt.plot(x,a.profiles[i]["neutral_fraction_O"], label="O fraction", color='y')
plt.plot(x,a.profiles[i]["neutral_fraction_N"], label="N fraction", color='g')
plt.plot(x,a.profiles[i]["neutral_fraction_Mg"], label="Mg fraction", color='k')
plt.plot(x,a.profiles[i]["neutral_fraction_Fe"], label="Fe fraction", color='purple')

plt.xlim(x[xmin],x[xmax])
plt.ylabel("Ionization Fraction")
plt.ylim(-0.05, 1.05)

plt.legend(loc=5)

plt.show()



# Now, let's plot the abundances of elements

fig, host = plt.subplots(figsize=(11.2,3))
fig.subplots_adjust(right=0.75)

plt.plot(x,np.log10(a.profiles[i]["h1"]), label="H")
plt.plot(x,np.log10(a.profiles[i]["he3"]), label="He3")
plt.plot(x,np.log10(a.profiles[i]["he4"]), label="He4")
plt.plot(x,np.log10(a.profiles[i]["c12"]), label="C12")
plt.plot(x,np.log10(a.profiles[i]["n14"]), label="N14")
plt.plot(x,np.log10(a.profiles[i]["o16"]), label="O16")

plt.xlim(xmin,xmax)
plt.ylabel("Elemental Abundances")
plt.ylim(-7.0, 0.0)

plt.legend(loc=2)

plt.show()




# Now, let's plot power sources and sinks and compare to luminosity

fig, host = plt.subplots(figsize=(11.2,3))
fig.subplots_adjust(right=0.75)
dm = a.profiles[i]["dm"]
Lsun = 3.828e33 #erg/s 
plt.plot(x,np.log10(a.profiles[i]["eps_grav"]*dm/Lsun), label="P_grav")
plt.plot(x,np.log10(a.profiles[i]["eps_nuc"]*dm/Lsun), label="P_nuc")
plt.plot(x,np.log10(a.profiles[i]["eps_recombination"]*dm/Lsun), label="P_rec")
plt.plot(x,np.log10(a.profiles[i]["eps_visc"]*dm/Lsun), label="P_visc")
plt.plot(x,a.profiles[i]["log_Lrad_div_L"]+a.profiles[i]["logL"], label="L_rad")
plt.plot(x,a.profiles[i]["logL"], label="L")

plt.xlim(xmin,xmax)
plt.ylabel("log(Power/L$_\odot$)")
plt.ylim(-10.0, 10)




plt.legend(loc=4)
plt.show()



# Now, let's plot energy componentsat each cell

fig, host = plt.subplots(figsize=(11.2,3))
fig.subplots_adjust(right=0.75)
dm = a.profiles[i]["dm"]
Lsun = 3.828e33 #erg/s 
G = 6.67259e-8
plt.plot(x,np.log10(np.abs(G*a.profiles[i]["mass"]/10**a.profiles[i]["logR"])), label="E_grav")
plt.plot(x,np.log10(np.abs(0.5*a.profiles[i]["velocity"]**2)), label="E_kin")
plt.plot(x,np.log10(np.abs(0.5*a.profiles[i]["v_rot"]**2)), label="E_rot")
plt.plot(x,np.log10(np.abs(a.profiles[i]["energy"])), label="E_int")
plt.plot(x,np.log10(np.abs(a.profiles[i]["ionization_energy"])), label="E_ion")
plt.plot(x,np.log10(np.abs(a.profiles[i]["total_energy"])), label="E_total")

plt.xlim(xmin,xmax)
plt.ylabel("log(abs(Specific Energy [erg/rg]))")
plt.ylim(12.0, 12.5)




plt.legend(loc=4)
plt.show()





In [ ]:
i = -1

plt.plot(a.profiles[i]['logRho'], a.profiles[i]['logT'])


xmax = a.profiles[i]['logRho'][np.argmax(a.profiles[i]['luminosity'][0:200]) ]
ymax = a.profiles[i]['logT'][np.argmax(a.profiles[i]['luminosity'][0:200]) ]
# xmax = a.profiles[i]['logRho'][95]
# ymax = a.profiles[i]['logT'][95]


plt.scatter(xmax, ymax, color='r')

print(xmax, ymax)

plt.plot([-10, -10, -9, -10], [3.5,3.6, 3.5, 3.5])

plt.xlim(-10.5, -8)
plt.ylim(3.3, 3.8)
plt.show()

In [ ]:
plt.plot(a.history["log_dt"])
plt.xlim(160,180)
plt.show()

In [ ]:
a.SetParameters(Variable='eps_nuc', cmap_dynamic_range=5, tau100=True, Yaxis='radius', Xaxis='star_age', czones=True)
a.Kippenhahn()

In [ ]:
a.SetParameters(cmap_dynamic_range=2.0, mass_locations_to_trace=[13.0,13.4,14, 15.0, 16.0, 18.0, 20.0,25.0, 29.0],signed_log_cmap=False,Yaxis='log_radius', tau10=True, tau100= True,Xaxis="star_age", czones=True, Variable='v_div_vesc')
a.Kippenhahn()
plt.savefig('sample.pdf')

In [ ]:
a.SetParameters(mass_locations_to_trace=[13.0,14, 15.0, 16.0, 18.0, 20.0,25.0, 29.0], cmap_dynamic_range=4.0, signed_log_cmap=True,Yaxis='radius', Xaxis="star_age", czones=True,tau10=True, tau100= True, Variable='eps_recombination')
a.Kippenhahn()

In [ ]:
a.SetParameters(Variable='total_energy', Xaxis="star_age", Yaxis="radius", cmap_dynamic_range=40,signed_log_cmap=True, tau100=True)
a.Kippenhahn()

In [ ]:

a.SetParameters(mass_locations_to_trace=[4.0,4.5, 5.0, 6.0, 8.0, 10.0,12.0, 14.0],cmap_dynamic_range=4, signed_log_cmap=False,Yaxis='radius', Xaxis="star_age", czones=True, Variable='v_div_vesc')
a.Kippenhahn()

In [ ]:
a.SetParameters(Variable='Lrad_div_Ledd', signed_log_cmap=False,Yaxis='radius', cmap_dynamic_range=3)
a.Kippenhahn()

In [ ]:
a.SetParameters(Xaxis="star_age", Yaxis="radius", Variable='t_thermal', cmap_dynamic_range=6, signed_log_cmap=False)
a.Kippenhahn()

In [ ]:
a.SetParameters(Yaxis="radius", Variable='t_dynamical', Xaxis="model_number", cmap_dynamic_range=5)
a.Kippenhahn()

In [ ]:
a.SetParameters(Yaxis="radius", Variable='t_thermal_div_t_dynamical', Xaxis="star_age", cmap_dynamic_range=6)
a.Kippenhahn()

In [ ]:
a.SetParameters(Yaxis="radius", Variable='opacity',Xaxis='star_age', cmap_dynamic_range=5, signed_log_cmap=False)
a.Kippenhahn()

In [ ]:
a.SetParameters(Variable='total_energy', Xaxis="star_age", Yaxis="radius", cmap_dynamic_range=5,signed_log_cmap=True, tau100=True)
a.Kippenhahn()

In [ ]:
plt.plot(a.history['star_age']*365, a.history["CE_energy_rate"], color='k')
plt.xlabel('Star Age (days)', size=20)
plt.ylabel(r'$L_{\rm CE}\ (erg/s)$', size=20)
plt.yscale('log')
#plt.ylim(1.0e38, 2.0e40)
plt.show()

In [ ]:
plt.plot(a.history['star_age']*365, 10**a.history["log_Teff"], color='k')
plt.xlabel("Star Age (yr)", size=20)
plt.ylabel(r'$T_{\rm eff}\ (K)$', size=20)
plt.show()

In [ ]:
plt.plot(a.history['star_age'], a.history["log_L"], color='k')
plt.xlabel("Star Age (yr)", size=20)
plt.ylabel(r"${\rm log}\ L\ (L_{\odot})$", size=20)
plt.show()

In [ ]:
plt.plot(a.history['log_Teff'], a.history["log_L"], color='k')
xwidth = max(a.history['log_Teff']) - min(a.history['log_Teff'])
plt.xlim(0.1*xwidth + max(a.history['log_Teff']), -0.1*xwidth + min(a.history['log_Teff']))
plt.xlabel(r"${\rm log}\ {\rm T}_{\rm eff}\ (K)$", size=20)
plt.ylabel(r"${\rm log}\ L\ (L_{\odot})$", size=20)
aa=1
bb=4
MM =16.
AA = 1./101.
BB = 7./51.
CC = 13./51.
DD = np.log10(4000)
logL_hayashi = np.linspace(np.min(a.history["log_L"]), np.max(a.history["log_L"]), num=20)
logTeff_hayashi = AA*logL_hayashi + BB* np.log10(MM) + CC*np.log10(0.5) + DD
print logL_hayashi
print logTeff_hayashi
plt.plot(logTeff_hayashi, logL_hayashi, color='g')
plt.show()

In [ ]:
secyer = 31622400.0

lum = 10.0**a.history['log_L']
time_step = 10.0**a.history['log_dt'] * secyer
time = a.history["star_age"]

E_rad = np.cumsum(lum*time_step)
plt.plot(time, E_rad * 3.839e33, color='k', label="Radiated away")


E_CE = np.cumsum(time_step*a.history["CE_energy_rate"])
plt.plot(time, E_CE, color='k', linestyle='dashed', label="Injected")


Msun = 1.988e33
G = 6.6725e-8
Rsun = 6.957e10
E_available = G*a.history['he_core_mass'][-1]*Msun*Msun/(0.34*Rsun)
print E_available


# plt.axhline(np.log10(E_available), color='k', linestyle='dotted', label="Available from orbit")
#plt.text("")

plt.xlabel('Star Age (yr)', size=16)
plt.ylabel('Cumulative Energy [erg]', size=16)

# plt.ylim(45, 48)


plt.legend(loc=4, fontsize=16)

plt.show()

In [ ]:
plt.plot(a.history["star_age"], np.log10(a.history['eta_pulse_wind']), color='k')
plt.xlabel("Star Age (yr)", size=20)
plt.ylabel(r"n pulsational wind ", size=20)
plt.show()

In [ ]:
plt.plot(a.history["model_number"], a.history['log_dt'], color='k')
plt.xlabel("Star Age (yr)", size=20)
plt.ylabel(r" log(dt/yr)", size=20)
plt.show()

In [ ]:
a.SetParameters(Yaxis="mass", Xaxis="model_number",Variable='j_rot', cmap_dynamic_range=5,signed_log_cmap=False)
a.Kippenhahn()

In [ ]:

a.SetParameters(Yaxis="mass", Variable='v_div_csound', Xaxis='star_age', cmap_dynamic_range=2.4,signed_log_cmap=False)
a.Kippenhahn()

In [ ]:
a.SetParameters(Yaxis="radius", Variable='density', cmap_dynamic_range=15,signed_log_cmap=False)
a.Kippenhahn()

In [ ]:
a.SetParameters(Yaxis="radius", Variable='pressure', cmap_dynamic_range=15,signed_log_cmap=False)
a.Kippenhahn()

In [ ]:
a.SetParameters(Yaxis="radius", Variable='temperature', cmap_dynamic_range=5,signed_log_cmap=False)
a.Kippenhahn()